# Re-Ranking

In the context of RAG (Retrieval-Augmented Generation), reranking of retrieval results is a crucial step that refines the initial set of retrieved documents based on their relevance to the input query. This process involves re-scoring the retrieved documents using a more sophisticated model, such as a cross-encoder, to better capture the semantic similarity between the query and the documents. The reranked list of documents is then used as input for the generation model, ensuring that the most relevant and accurate information is utilized to generate the final output.

![Cross Encoder Image](https://raw.githubusercontent.com/UKPLab/sentence-transformers/master/docs/img/CrossEncoder.png)


https://www.sbert.net/examples/applications/retrieve_rerank/README.html

In [1]:
from rich.console import Console
from rich_theme import custom_theme

# Create a console with the dark theme
console = Console(theme=custom_theme)


In [2]:
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

In [3]:
from sentence_transformers import CrossEncoder 
cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

## Loading retrival results

We will load the retrival results from the previous Hybrid-Search notebook, to avoid repeatition.

In [4]:
import json
retrieved_documents = json.load(open('data/retrieved_docs.json'))

In [17]:
console.print(retrieved_documents)

[
    {
        'id': 15,
        'text': "3.8 â Mixtral_8x7B 3.5 32 > $3.0 i] 228 fos a 2.0 0 5k 10k 15k 20k 25k 30k Context length Passkey 
Performance ry 3.8 â Mixtral_8x7B 3.5 0.8 32 > 0.6 $3.0 i] 228 04 fos 0.2 a 2.0 0.0 OK 4K 8K 12K 16K 20K 24K 28K 0 
5k 10k 15k 20k 25k 30k Seq Len Context length Figure 4: Long range performance of Mixtral. (Left) Mixtral has 100% 
retrieval accuracy of the Passkey task regardless of the location of the passkey and length of the input sequence. 
(Right) The perplexity of Mixtral on the proof-pile dataset decreases monotonically as the context length 
increases.\n\nThe chunk discusses the long-range performance of the Mixtral model, demonstrating its ability to 
retrieve a passkey regardless of its location in a long input sequence, and showing that the model's perplexity on 
the proof-pile dataset decreases as the context length increases.",
        'dense_score': 0.6180975806351034,
        'sparse_score': '1.4473938'
    },
    {
        'id': 4,
        'text': "Instruct under the Apache 2.0 license1, free for academic and commercial usage, ensuring broad 
accessibility and potential for diverse applications. To enable the community to run Mixtral with a fully 
open-source stack, we submitted changes to the vLLM project, which integrates Megablocks CUDA kernels for efficient
inference. Skypilot also allows the deployment of vLLM endpoints on any instance in the cloud. # 2 Architectural 
details Mixtral is based on a transformer architecture [31] and uses the same modifications as described in [18], 
with the notable exceptions that Mix- tral supports a fully dense context length of 32k tokens, and the feed- 
forward blocks are replaced by Mixture-of-Expert layers (Section 2.1). The model architecture parameters are 
summarized in Table 1.\n\nThis chunk describes the architectural details of the Mixtral language model, including 
its use of a transformer architecture with a 32k token context length and mixture-of-expert layers. It also 
mentions the model's open-source licensing and deployment options.",
        'dense_score': 0.4972174713599332,
        'sparse_score': '1.1622587'
    },
    {
        'id': 2,
        'text': 'expertsâ ) to process the token and combine their output additively. This technique increases the 
number of parameters of a model while controlling cost and latency, as the model only uses a fraction of the total 
set of parameters per token. Mixtral is pretrained with multilingual data using a context size of 32k tokens. It 
either matches or exceeds the performance of Llama 2 70B and GPT-3.5, over several benchmarks. In particular, 
Mixture of Experts Layer i gating inputs af outputs router expert\n\nThis chunk describes the key architectural 
details of the Mixtral model, a sparse mixture-of-experts language model that outperforms larger models like Llama 
2 70B and GPT-3.5 on various benchmarks.',
        'dense_score': 0.44239135249907124,
        'sparse_score': '2.1099076'
    },
    {
        'id': 6,
        'text': 'Table 1: Model architecture. # j nâ G(x)i Â· Ei(x). i=0 Here, G(x)i denotes the n-dimensional 
output of the gating network for the i-th expert, and Ei(x) is the output of the i-th expert network. If the gating
vector is sparse, we can avoid computing the outputs of experts whose gates are zero. There are multiple 
alternative ways of implementing G(x) [6, 15, 35], but a simple and performant one is implemented by taking the 
softmax over the Top-K logits of a linear layer [28].\n\nThe chunk describes the architectural details of the 
Mixtral model, specifically the Sparse Mixture of Experts (SMoE) layer that is used in the model.',
        'dense_score': 0.4404994080056008
    },
    {
        'id': 7,
        'text': 'We use G(x) := Softmax(TopK(x Â· Wg)), where (TopK(â ))i := â i if â i is among the top-K 
coordinates of logits â â Rn and (TopK(â ))i := â â otherwise. The value of K â the number of experts used per 
token â is a hyper-p

In [20]:
# This is the query that we used for the retrieval of the above documents
query = "What is context size of Mixtral?"


## Calculating the re-ranking scores

In [19]:
pairs = [[query, doc['text']] for doc in retrieved_documents] 
scores = cross_encoder.predict(pairs) 

console.print(scores)

[ 5.065694   3.368832   7.1048393 -4.116105  -4.3754997 -5.2610755
 -3.7225761  3.1854553  1.7966702 -2.5144258]

## Selecting top 3 reranked documents

In [16]:
import numpy as np
print("Top 3 Documents:") 
for o in np.argsort(scores)[::-1][:3]:
    print(retrieved_documents[o]['text'])

Top 3 Documents:
expertsâ ) to process the token and combine their output additively. This technique increases the number of parameters of a model while controlling cost and latency, as the model only uses a fraction of the total set of parameters per token. Mixtral is pretrained with multilingual data using a context size of 32k tokens. It either matches or exceeds the performance of Llama 2 70B and GPT-3.5, over several benchmarks. In particular, Mixture of Experts Layer i gating inputs af outputs router expert

This chunk describes the key architectural details of the Mixtral model, a sparse mixture-of-experts language model that outperforms larger models like Llama 2 70B and GPT-3.5 on various benchmarks.
3.8 â Mixtral_8x7B 3.5 32 > $3.0 i] 228 fos a 2.0 0 5k 10k 15k 20k 25k 30k Context length Passkey Performance ry 3.8 â Mixtral_8x7B 3.5 0.8 32 > 0.6 $3.0 i] 228 04 fos 0.2 a 2.0 0.0 OK 4K 8K 12K 16K 20K 24K 28K 0 5k 10k 15k 20k 25k 30k Seq Len Context length Figure 4: Long range p